In [36]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from env import *
from time import sleep
import prepare

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [2]:
# names = pd.read_csv('more_players.csv')

In [3]:
# names = names [['players']]

In [4]:
# names

In [5]:
# names[names.index==85]

In [6]:
# names

In [7]:
def get_puuid(name_list):
    puuid_list = []
    counter = 0
    for name in name_list:
        print(counter)
        counter += 1
        if counter%50 == 0:
            print('function paused')
            print
            time.sleep(180)
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
    return puuid_list

In [8]:
# puuid_list = get_puuid(names.players)

In [9]:
# puuid_list

In [10]:
# puuid_df = pd.DataFrame(puuid_list)

In [11]:
# puuid_df.drop(columns = 'status', inplace=True)

In [12]:
# puuid_df.dropna(inplace=True)

In [13]:
# puuid_df.to_csv('puuid.csv',index=False)

In [14]:
# puuid = puuid_df[['puuid']]

In [15]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

In [16]:
#match_ids = pd.read_csv('match_ids.csv')

In [17]:
#match_ids = list(match_ids['0'])[1000:2000]

In [18]:
# df = get_match_info(match_ids, api_key, username, time=20)

In [19]:
#other_game_data = pd.read_json('other_game_data_bryant.json')

In [20]:
#timeline_data = pd.read_json('timeline_data_bryant.json')

In [21]:
#timeline_list = timeline_data.to_dict(orient = 'records')

In [22]:
#other_game_list = other_game_data.to_dict(orient = 'records')

In [23]:
# df = prepare(timeline_list, other_game_list, time=20)

In [24]:
# df.head()

In [25]:
# df.size

In [26]:
# df.shape

In [27]:
# df.columns

In [28]:
# df.to_csv('bryant_games.csv')

In [29]:
# df.winningTeam.value_counts()

In [30]:
#Load the match_ids.csv file
matches_df = pd.read_csv('match_ids.csv')
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4094915650
3,NA1_4082128299
4,NA1_4083500908
...,...
37279,NA1_4097058147
37280,NA1_4096721229
37281,NA1_4094049450
37282,NA1_4091691873


In [31]:
#Set up the match id list. You only need to uncomment the line following your name

match_ids = list(matches_df['0'])[10_657:17_314]

In [32]:
#Set up your username. You only need to uncomment the line following your name.
#Again, this username is only for naming the json files.

#Joshua Bryant
username = 'bryant'

In [33]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

In [34]:
#Before calling the get_match_info function, separate the match_ids list into smaller batches
match_batch_1 = match_ids[:1000]
match_batch_2 = match_ids[1000: 2000]
match_batch_3 = match_ids[2000: 3000]
match_batch_4 = match_ids[3000: 4000]
match_batch_5 = match_ids[4000: 5000]
match_batch_6 = match_ids[5000: 6000]
match_batch_7 = match_ids[6000: 6657]

In [35]:
#Since I will be running this function in smaller batches, I will have to update the username
#after each iteration. This will allow me to keep track of which json files contain what data

#For first iteration
username = 'start_10_657_end_11_657'

#Call the get_match_info function
df1 = get_match_info(match_batch_1, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

AttributeError: 'function' object has no attribute 'prepare'